# House Prices Attempt I: Linear Regression

For my first non-physics machine learning project I will be testing out various techniques on a straightforward house prices dataset. The first attempt will be a simple linear regression. The plan is to use one hot encoding (etc.) to deal with the categorical data which should be easy to do with the **pandas** toolset. I also expect some ordinal features will not lend well to a simple linear model, so I will also try making up some nonlinear features (*e.g.* (Overall Quality)^2).  I expect that the number of features may become unwieldly, but we will see :).

Standard imports:

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd

Load raw training data:

In [4]:
data = pd.read_csv("../data/train.csv")
print(data.shape)

(1460, 81)


OK, so there are $m = 1168$ training examples and $n = 80$ features (ignoring 'Id'). So I'm pretty sure after one hot encoding $n$ will be approaching $m$. This may be bad but I'm going to allow it just to see what happens.

## One hot encoding
Here are the categorical features that I have decided to encode:

In [5]:
ohe_categories = [
    'MSZoning',
    'Street',
    'Alley',
    'LotShape',
    'LandContour',
    'LotConfig',
    'Neighborhood',
    'Condition1',
    'Condition2',
    'BldgType',
    'HouseStyle',
    'RoofStyle',
    'RoofMatl',
    'Exterior1st',
    'Exterior2nd',
    'MasVnrType',
    'Foundation',
    'Heating',
    'Electrical',
    'GarageType',
    'MiscFeature',
    'SaleType',
    'SaleCondition'
]
data_augmented_1 = pd.get_dummies(data[ohe_categories])
data_augmented_1.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0


I will add the treated data to the *data_augmented* variable as I go.

## Ordinal data mapping

I will use the **pd.DataFrame.cat** functionality to map the ordinal data to numbers. I will just map them as natural numbers for now. Here are the categories that will need maping.

In [6]:
odm_ordered_lists = {
    'Utilities': ['ELO', 'NoSeWa', 'NoSewr', 'AllPub'],
    'LandSlope': ['Sev', 'Mod', 'Gtl'],
    'ExterQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'ExterCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'BsmtExposure': ['No', 'Mn', 'Av', 'Gd'],
    'BsmtFinType1': ['Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    'BsmtFinType2': ['Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    'HeatingQC': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'CentralAir': ['N','Y'],
    'KitchenQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'], 
    'Functional': ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
    'FireplaceQu': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'GarageFinish': ['Unf', 'RFn','Fin'],
    'GarageQual': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'GarageCond': ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    'PavedDrive': ['Y', 'P', 'N'],
    'PoolQC': ['Fa', 'TA', 'Gd', 'Ex'],
    'Fence': ['MnWw', 'GdWo', 'MnPrv', 'GdPrv'],    
}

def index_exclude_nan(ordered_list,string):
    """The NaN's will be set to 0 since not having something is probably bad.
       Also add 1 to the indices so they are always natural numbers."""
    if isinstance(string, str):
        return ordered_list.index(string)+1
    else:
        return 0
        
def odm(series,ordered_list):
    "Quick and dirty way to map series entry to ordered_list index"
    return pd.Series(list(map(lambda x: index_exclude_nan(ordered_list,x),series)),index = series.index)

data_augmented_2 = pd.DataFrame()
for cat in odm_ordered_lists:
    data_augmented_2[cat] = odm(data[cat],odm_ordered_lists[cat])

data_augmented_2.head()

,Utilities,LandSlope,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,CentralAir,KitchenQual,Functional,FireplaceQu,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence
0,4,3,4,3,4,3,1,6,1,5,2,4,8,0,2,3,3,1,0,0
1,4,3,3,3,4,3,4,5,1,5,2,3,8,3,2,3,3,1,0,0
2,4,3,4,3,4,3,2,6,1,5,2,4,8,3,2,3,3,1,0,0
3,4,3,3,3,3,4,1,5,1,4,2,4,8,4,1,3,3,1,0,0
4,4,3,4,3,4,3,3,6,1,5,2,4,8,3,2,3,3,1,0,0


## Clean up remaining data

The numerical data can now be cleaned up by replacing NaN's with the average value of the column

In [7]:
unwanted_categories = ['Id','SalePrice','MasVnrType','MasVnrArea']
def is_used_category(cat):
    "Find all categorical and unwanted categories."
    return np.array([(c in ohe_categories) or (c in odm_ordered_lists) or (c in unwanted_categories) for c in cat])

def nan_to_avg(series):
    "Replace nan with series average."
    return series.fillna(np.mean(series))

data_augmented_3 = data.loc[:, np.invert(is_used_category(data.columns))]

data_augmented_3 = data_augmented_3.apply(nan_to_avg)

data_augmented_3.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,706,0,150,...,548,0,61,0,0,0,0,0,2,2008
1,20,80.0,9600,6,8,1976,1976,978,0,284,...,460,298,0,0,0,0,0,0,5,2007
2,60,68.0,11250,7,5,2001,2002,486,0,434,...,608,0,42,0,0,0,0,0,9,2008
3,70,60.0,9550,7,5,1915,1970,216,0,540,...,642,0,35,272,0,0,0,0,2,2006
4,60,84.0,14260,8,5,2000,2000,655,0,490,...,836,192,84,0,0,0,0,0,12,2008


Note: After running the linear regression with a few random cross-validation test sets, I found that the cross validtion error will diverge if the masonry veneer type is None for all the training examples. The quickest solution is just to ignore the features 'MasVnrType' and 'MasVnrArea'. Retaining two features is not a high priority for this small project.

Now combine all augmented datasets into the final training dataset.

In [9]:
data_augmented = pd.concat([data_augmented_1,data_augmented_2,data_augmented_3],axis = 1)
data_augmented.to_csv('../data-augmented/train_augmented.csv')
data_augmented.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0,0,0,1,0,0,1,0,0,0,...,548,0,61,0,0,0,0,0,2,2008
1,0,0,0,1,0,0,1,0,0,0,...,460,298,0,0,0,0,0,0,5,2007
2,0,0,0,1,0,0,1,0,0,1,...,608,0,42,0,0,0,0,0,9,2008
3,0,0,0,1,0,0,1,0,0,1,...,642,0,35,272,0,0,0,0,2,2006
4,0,0,0,1,0,0,1,0,0,1,...,836,192,84,0,0,0,0,0,12,2008


In [7]:
data_augmented.shape

(1460, 223)

Fortunately, the number of features didn't become as large as I had guessed. Now I can move on to the linear regression.

## Linear Regression

In [9]:
from sklearn.linear_model import LinearRegression

X = data_augmented
y = data['SalePrice'][data_augmented.index]
cv_split = 4

model = LinearRegression()
model.fit(X,y);

In [10]:
from sklearn.model_selection import cross_val_score
cv_score = cross_val_score(model, X, y, cv=cv_split)
print("Accuracy: %0.2f (+/- %0.2f)" % (cv_score.mean(), cv_score.std() * 2))

Accuracy: 0.80 (+/- 0.13)


So this cross validation (CV) accuracy value means that the model accounts for about 80% of the variance in the CV dataset. Not too bad for one of the simplest models I could think of, but not exactly ready to ship. Now I can adjust the model to try and maximize the CV score. First step is to add regularization by using a ridge regressor instead of a plain linear model, then optimize the regulraization paramater *alpha* to achieve the best cross validation score.

In [12]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [0, 0.03, .1, .3, 1, 3, 10, 30, 100, 300]}

grid = GridSearchCV(Ridge(),param_grid, cv = cv_split)
grid.fit(X,y)
model = grid.best_estimator_
cv_score = cross_val_score(model, X, y, cv=cv_split)
print("Accuracy: %0.2f (+/- %0.2f)" % (cv_score.mean(), cv_score.std() * 2))

Accuracy: 0.83 (+/- 0.14)
